#### Pre-Processing

In [ ]:
# Parse a PDF as well as a email invoice
from src.utils import preprocess_aor, preprocess_invoice
# pdf_path = "database/aor/AOR-sample1.pdf"
# preprocess_aor(aor_dir="database/aor")

# preprocess_invoice(invoice_dir="database/invoice")

#### Search & Query

In [ ]:
# Chat with the preprocessed AOR for real-time chat based on RAG 
from src.aor import load_aors
from src.search import search_aor_with_item, search_aor_with_no


aor_list = load_aors()

# Quick routing: Item-based & Number-baseds
no_query = "DH928"
matching_aors = search_aor_with_no(no_query, aor_list)


print("Item-based AOR searching:")
for aor in matching_aors[:1]:
    print(f"AOR No: {aor.no}")
    print(f"Description: {aor.description}")
    print(f"Items: {aor.items}")
    print(f"Budgets: {aor.budgets}")
    print(f"Expiry Date: {aor.expiry_date}")
    print("---")

#### RAG-based chat 

In [ ]:
from src.agent import RagAgent 

# Example usage
agent = RagAgent()
user_message = "Tell me about AOR DH928"
agent_response = agent.chat(user_message)

user_message = "What is the invoice no.01141833 about?"
agent_response = agent.chat(user_message)

user_message = "Is this invoice covered by the AOR?"
agent_response = agent.chat(user_message)

In [15]:
from src.utils import *
import glob 

eml_files = glob.glob("database/aor/*.eml")
eml_file_path = eml_files[0]
email_data = read_eml(eml_file_path)

email_txt = email_data.get('body')
prompt = PARSE_AOR_PROMPT.format(pdf_txt=email_txt)
response = get_oai_response(prompt)
parsed_aor_dict = parse_json_response(response)
aor = AOR(**parsed_aor_dict)


# print(email_data)

In [13]:
# type(email_data)
email_data['body']

"Message Classification: Restricted\r\n\r\nApproved. Tks.\r\n\r\nBest,\r\nHDM\r\n\r\nFrom: Seow Yuxin <yuxin.seow@temus.com>\r\nDate: Thursday, 15 August 2024 at 5:01\u202fPM\r\nTo: Yu Fangyuan <Fangyuan.Yu@temus.com>\r\nSubject: [SAMPLE AOR] [For HDM's approval] APPROVAL OF REQUIREMENT FOR CAMP 1 OPS COST REQUIREMENTS [Restricted]\r\nMessage Classification: Restricted\r\n\r\nHDM sir,\r\n\r\nAPPROVAL OF REQUIREMENT FOR  OPS COST REQUIREMENTS FOR CAMP 1\r\n\r\nAIM:\r\n1. To seek HDM’s approval for Ops Cost requirements at an estimated cost of S$16,290 (26.56THB to 1 SGD)  – MAS Ex Rate for 2024.\r\n\r\nCOST BREAKDOWN AND JUSTIFICATION\r\n2. Approval is sought for the following requirements:\r\n\r\nItems\r\nForecast for 2022\r\n2022 Actual Expenditure (THB)\r\nForecast for FY23/24 (THB)\r\nRemarks\r\nTelco Subscription\r\n               339,488.00\r\n                                               282,907.00\r\n                                       424,361.00\r\nProjected amount derived 

In [ ]:
from PIL import Image
import glob
# glob.glob("assets/*")[1]
icon_path = "assets/flow-agent.png"
icon = Image.open(icon_path)

#### Invoice Processor

In [ ]:
# this is the last query to solve, before the front-end deployment 
# query = "Is the invoice no.01141833 covered by any of the AORs?"
# query = "What is the invoice no.01141883 about?"

# use_message = "Is the invoice no.01141833 covered by any of the AORs?"
agent_response = agent.chat("Is the invoice no.01141833 covered by any of the AORs?")

In [ ]:
# Chainlit front-end interface with logo change now
